### Stock Market Prediction And Forecasting Using Stacked LSTM

### Stock Price Prediction by using L.S.T.M  (univariate)
1. #### Preparing the most Recent data for stock price prediction
2. #### Understanding L.S.T.M
3. #### Preparing the data in L.S.T.M format (with 60 days lookback)
4. #### Creating L.S.T.M netword using Keras 
5. #### Creating a Training and Test and Validation Data
6. #### Future prediction for 30 upcoming days 

##### Author - Abhishek Jaiswal

In [ ]:
# importing the necessary library
%config Completer.use_jedi = False # if autocompletion doesnot work in kaggle notebook | hit tab
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore') # igoring any kind of warning if comes 


In [ ]:
import pandas_datareader as web
df = web.DataReader('AAPL', data_source='yahoo',start = '2013-01-01',end = '2018-01-01')
df

In [ ]:
df.head()

In [ ]:
df.info()

From here we can see our data has 6 columns:

Date: The date of recorded data

Open: The price when stock market open

High: The highest price price of date

Low: The lowest price point of date

Volumn: Total Sale of stock on that date

From this point we going to define our goal is to predict the close price of the given date from test data

### Visualisation 

In [ ]:
fig, ax = plt.subplots(figsize = (15,5))
ax.plot(df['Open'])
ax.set_title("Google Opening Prices")
ax.set_xlabel("Time (oldest -> latest)")
ax.set_ylabel("Stock Opening Price")
plt.show()


In [ ]:

plt.figure(figsize=(18, 8))
plt.plot(df['Close'])
plt.title("Google Closing Prices")
plt.xlabel("Time (oldest-> latest)")
plt.ylabel("Stock Hightest Points")
plt.show()
# here between  graphs we have 180 days gap


In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(df['Volume'])
plt.title("Volume of stocks sold")
plt.xlabel("Time (oldest-> latest)")
plt.ylabel("Volume of stocks traded")
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,5))
sns.lineplot(data=df[['High','Low']],linewidth=2)
plt.grid(True)
plt.show

In [ ]:
len(df)

### Here we have to predict closing price so first we will train closing price sequence into lstm

In [ ]:
data = df['Close'].values
len(data)

In [ ]:
# splitting the data for training and prediction purpose 
train_len = int(np.ceil(len(data)*0.9))
train_data = data[0:train_len]
len(train_data)

#### Scaling the data by using min max scaler 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train_data_scaled = scaler.fit_transform(train_data.reshape(-1,1))


In [ ]:
# so far we have scaled our data for training and left one part for prediction 
# now lets prepare the data for lstm format 
# now lets make a lookup data for rnn with 100 days lookback 
# creating a lookback data of 100 days
X_input = []
y_input = []

for i in range(100, len(train_data_scaled)):
    X_input.append(train_data_scaled[i-100:i,0])
    y_input.append(train_data_scaled[i,0])

In [ ]:
X_input,y_input = np.array(X_input),np.array(y_input) # converting into arrays 
X_input.shape

#### from training data we are taking out 20 samples for validation data which we gonna use while training our model

In [ ]:
X_train = X_input[20:]
y_train = y_input[20:]
X_valid = X_input[:20] # validation data of 10 values 
y_valid = y_input[:20] # validation data of 10 samples which will be used while training

In [ ]:
print(X_train.shape,X_valid.shape)

### Here training data is in 2D we have to convert into 3D for R.N.N lstm
3rd axis will be the number of features are we taking

In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1)) # 1 means 1 feature we are taking
X_valid = np.reshape(X_valid,(X_valid.shape[0],X_valid.shape[1],1))
print(X_train.shape,X_valid.shape)

### Now lets create L.S.T.M model by using keras

In [ ]:
# now creating L.S.T.M 
from keras import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(50, return_sequences= True, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=True, activation='relu'))
model.add(LSTM(50))
model.add(Dense(1))
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_valid, y_valid), callbacks=callbacks)


## Now lets predict by our model 
before prediction we have to prepare our testing data which will be again with past 60 days lookback

Steps involving for prediction of test data

Here input data is just a past 100 days history 

we will need past 100 days to predict one future day


#### lets suppose we have no future data and only we have past 100 days data and on the basic of past data we will predict future

In [ ]:
# lets first predict for single point daata
len(data[:train_len][-100:]) # this is how we can access last 100 days from our training data 

In [ ]:
test_point =  train_data_scaled[-100:]

In [ ]:
X_test_point = np.array(test_point).reshape(1,100,1)

In [ ]:
model.predict(X_test_point)

In [ ]:
y_predicted_point = scaler.inverse_transform(model.predict(X_test_point))
y_predicted_point

#### Testing on the test data


#### for one data prediction of future we need 60 days of past 
#### if we will predict one day by 60 days and if we try to predict more days on the past 60 days error will get increased over prediction

In [ ]:
test_data  =  data[train_len-100:]
test_data_scaled = scaler.transform(np.array(test_data).reshape(-1,1))
X_test = []
for i in range(100,len(test_data_scaled)):
    X_test.append(test_data_scaled[i-100:i,0])
X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))

In [ ]:
prediction = scaler.inverse_transform(model.predict(X_test))

In [ ]:

#Plot the data
train = df[:train_len]
test = df[train_len:]
test['Prediction'] = prediction
# visualisation 
plt.figure(figsize = (16,8))
plt.title('Model')
plt.ylabel('Close Price USD', fontsize = 18)
plt.plot(train['Close'])
plt.plot(test[['Close','Prediction']])
plt.legend(['Train','Test','Prediction'], loc = 'lower right')
plt.show()


In [ ]:
plt.figure(figsize = (16,8))
plt.title('Model')
plt.plot(test[['Close','Prediction']])
plt.legend(['Test','Prediction'], loc = 'lower right')
plt.show()
